In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from joblib import dump, load
import gzip
import numpy as np

In [2]:
df = pd.read_parquet('./data/ETL.parquet')
nombres = pd.read_parquet('./data/NombresJuegos.parquet')
# Selecciona las características relevantes para el modelo
features = df.columns.values.tolist()


In [3]:
df

,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_2D,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Atmospheric,...,Etiquetas_Free to Play,Etiquetas_Great Soundtrack,Etiquetas_Indie,Etiquetas_Multiplayer,Etiquetas_Platformer,Etiquetas_Puzzle,Etiquetas_RPG,Etiquetas_Simulation,Etiquetas_Singleplayer,Etiquetas_Strategy
0,9,282010,1997,1,1,9319,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,9,282010,1997,1,1,9319,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,9,70,1998,1,2,2650946,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33539,9,80,2004,1,0,2469131,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
33540,9,80,2004,1,1,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
33541,9,80,2004,0,1,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
33542,9,80,2004,1,2,2469131,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
features = df.drop(columns=['ItemId'])

In [5]:
features

,Precio,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_2D,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Atmospheric,Etiquetas_Casual,Etiquetas_Free to Play,Etiquetas_Great Soundtrack,Etiquetas_Indie,Etiquetas_Multiplayer,Etiquetas_Platformer,Etiquetas_Puzzle,Etiquetas_RPG,Etiquetas_Simulation,Etiquetas_Singleplayer,Etiquetas_Strategy
0,9,1997,1,1,9319,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,1997,1,1,9319,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,9,1997,1,1,9319,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,9,1997,1,1,9319,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,9,1998,1,2,2650946,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33539,9,2004,1,0,2469131,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
33540,9,2004,1,1,2469131,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33541,9,2004,0,1,2469131,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
33542,9,2004,1,2,2469131,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33544 entries, 0 to 33543
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Precio                      33544 non-null  int64
 1   Año_Lanzamiento             33544 non-null  int64
 2   Recomendado                 33544 non-null  int64
 3   Sentimiento                 33544 non-null  int64
 4   TiempoJugado                33544 non-null  int64
 5   Etiquetas_2D                33544 non-null  int64
 6   Etiquetas_Action            33544 non-null  int64
 7   Etiquetas_Adventure         33544 non-null  int64
 8   Etiquetas_Atmospheric       33544 non-null  int64
 9   Etiquetas_Casual            33544 non-null  int64
 10  Etiquetas_Free to Play      33544 non-null  int64
 11  Etiquetas_Great Soundtrack  33544 non-null  int64
 12  Etiquetas_Indie             33544 non-null  int64
 13  Etiquetas_Multiplayer       33544 non-null  int64
 14  Etique

In [7]:
# Preprocesamiento y normalización de datos
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(features)

In [8]:

# Cálculo de la similitud de coseno
similarities = cosine_similarity(df_normalized)

In [9]:
np.save('matriz_similitud.npy', similarities)

In [2]:
data = np.load('matriz_similitud.npy')

In [6]:
data

array([[1.        , 0.57114104, 0.57114104, ..., 0.21297024, 0.59713585,
        0.49936136],
       [0.57114104, 1.        , 0.57114104, ..., 0.21297024, 0.59713585,
        0.49936136],
       [0.57114104, 0.57114104, 1.        , ..., 0.21297024, 0.59713585,
        0.49936136],
       ...,
       [0.21297024, 0.21297024, 0.21297024, ..., 1.        , 0.79504668,
        0.68647119],
       [0.59713585, 0.59713585, 0.59713585, ..., 0.79504668, 1.        ,
        0.83413657],
       [0.49936136, 0.49936136, 0.49936136, ..., 0.68647119, 0.83413657,
        1.        ]])

In [8]:
a = pd.DataFrame(data)

In [10]:
a.to_parquet('similitud.parquet', index=False)

In [9]:
similarities

array([[1.        , 0.57114104, 0.57114104, ..., 0.21297024, 0.59713585,
        0.49936136],
       [0.57114104, 1.        , 0.57114104, ..., 0.21297024, 0.59713585,
        0.49936136],
       [0.57114104, 0.57114104, 1.        , ..., 0.21297024, 0.59713585,
        0.49936136],
       ...,
       [0.21297024, 0.21297024, 0.21297024, ..., 1.        , 0.79504668,
        0.68647119],
       [0.59713585, 0.59713585, 0.59713585, ..., 0.79504668, 1.        ,
        0.83413657],
       [0.49936136, 0.49936136, 0.49936136, ..., 0.68647119, 0.83413657,
        1.        ]])

In [1]:
import numpy as np
import gzip

# Cargar el archivo .npy
data = np.load('matriz_similitud.npy')

# Crear un archivo gzip
with gzip.open('matriz_similitud.npy.gz', 'wb') as f:
    # Guardar los datos comprimidos
    np.save(f, data)

In [10]:
# Función para obtener los ítems más similares dado un ítem de referencia
def obtener_similares(id_item_referencia, n=5):
    index_referencia = df[df['ItemId'] == id_item_referencia].index[0]
    similar_indices = similarities[index_referencia].argsort()[::-1][:n+1]
    similar_items = df.iloc[similar_indices]
    return similar_items

In [11]:
id_item_referencia = 80  # Id del ítem de referencia
items_similares = obtener_similares(id_item_referencia)
print(items_similares["ItemId"])

33516        80
312        4760
25242    247430
33268       320
33503     13230
33282       220
Name: ItemId, dtype: int64


In [11]:
items_similares

,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_2D,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Atmospheric,...,Etiquetas_Free to Play,Etiquetas_Great Soundtrack,Etiquetas_Indie,Etiquetas_Multiplayer,Etiquetas_Platformer,Etiquetas_Puzzle,Etiquetas_RPG,Etiquetas_Simulation,Etiquetas_Singleplayer,Etiquetas_Strategy
33516,9,80,2004,1,1,2469131,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
312,9,4760,2004,1,1,3007902,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25242,6,247430,2004,1,1,44616,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33268,4,320,2004,1,1,2832582,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33503,14,13230,2004,1,1,345093,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
33282,9,220,2004,1,1,10719468,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:

items_similares['ItemId'] = items_similares['ItemId'].astype(str)
df_resultado = pd.merge(items_similares, nombres, on='ItemId')
df_resultado

/tmp/ipykernel_23921/4229142044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_similares['ItemId'] = items_similares['ItemId'].astype(str)


,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_2D,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Atmospheric,...,Etiquetas_Great Soundtrack,Etiquetas_Indie,Etiquetas_Multiplayer,Etiquetas_Platformer,Etiquetas_Puzzle,Etiquetas_RPG,Etiquetas_Simulation,Etiquetas_Singleplayer,Etiquetas_Strategy,Titulo
0,9,80,2004,1,1,2469131,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Counter-Strike: Condition Zero
1,9,4760,2004,1,1,3007902,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Rome: Total War™ - Collection
2,6,247430,2004,1,1,44616,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Hitman: Contracts
3,4,320,2004,1,1,2832582,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Half-Life 2: Deathmatch
4,14,13230,2004,1,1,345093,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Unreal Tournament 2004: Editor's Choice Edition
5,9,220,2004,1,1,10719468,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Half-Life 2


: 